## PySAL Change Log Statistics: Table Generation

This notebook generates the summary statistics for use in the 6-month releases of PySAL, which is now a meta package. 

It assumes the subpackages have been git cloned in a directory below the location of this notebook. It also requires network connectivity for some of the reporting.

Run this notebook after `100-gitcount.ipynb`


In [143]:
from __future__ import print_function
import os
import json
import re
import sys
import pandas
import subprocess
from subprocess import check_output

#import yaml
from datetime import datetime, timedelta

from dateutil.parser import parse
import pytz

utc=pytz.UTC

try:
    from urllib import urlopen
except:
    from urllib.request import urlopen




In [144]:
PYSAL_RELEASE = '2021-07-31' # date of this release
release_date = datetime.strptime(PYSAL_RELEASE+" 0:0:0", "%Y-%m-%d %H:%M:%S")

In [145]:
release_date

datetime.datetime(2021, 7, 31, 0, 0)

In [146]:
CWD = os.path.abspath(os.path.curdir)

In [147]:
CWD

'/Users/serge/Dropbox/p/pysal/src/pysal/tools'

In [148]:
start_date = '2021-01-31' # date of previous meta release
since_date = '--since="{start}"'.format(start=start_date)
since_date
since = datetime.strptime(start_date+" 0:0:0", "%Y-%m-%d %H:%M:%S")
since

datetime.datetime(2021, 1, 31, 0, 0)

In [149]:
since_date

'--since="2021-01-31"'

In [150]:
with open('frozen.txt', 'r') as package_list:
    packages = package_list.readlines()
    packages = dict([package.strip().split(">=") for package in packages])

In [151]:
packages

{'libpysal': '4.5.1.post2',
 'access': '1.1.3',
 'esda': '2.4.1',
 'giddy': '2.3.3',
 'inequality': '1.0.0',
 'pointpats': '2.2.0',
 'segregation': '2.0.0',
 'spaghetti': '1.6.2',
 'mgwr': '2.1.2',
 'spglm': '1.0.8',
 'spint': '1.0.7',
 'spreg': '1.2.4',
 'spvcm': '0.3.0',
 'tobler': '0.8.2',
 'mapclassify': '2.4.3',
 'splot': '1.1.4',
 'spopt': '0.1.2'}

In [152]:
import pysal
packages['pysal'] = pysal.__version__

In [153]:
import pickle

In [154]:
issues_closed = pickle.load(open("issues_closed.p", 'rb'))
pulls_closed = pickle.load(open('pulls_closed.p', 'rb'))

In [155]:
type(issues_closed)

dict

In [156]:
issues_closed.keys()

dict_keys(['libpysal', 'access', 'esda', 'giddy', 'inequality', 'pointpats', 'segregation', 'spaghetti', 'mgwr', 'spglm', 'spint', 'spreg', 'spvcm', 'tobler', 'mapclassify', 'splot', 'spopt', 'pysal'])

In [157]:
from release_info import get_pypi_info, get_github_info, clone_masters

In [158]:
#github_releases = get_github_info()

github_releases = pickle.load(open("releases.p", 'rb'))


In [159]:
pypi_releases = get_pypi_info()

In [160]:
from datetime import datetime

In [161]:
pysal_date = datetime.strptime('2021-07-31T12:00:00Z', '%Y-%m-%dT%H:%M:%SZ')
#ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


In [162]:
pysal_rel = {'version': 'v2.5.0',
            'release_date': pysal_date}
github_releases['pysal'] = pysal_rel

In [163]:
github_releases

{'libpysal': {'version': 'v4.5.1.post2',
  'url': 'https://api.github.com/repos/pysal/libpysal/tarball/v4.5.1.post2',
  'release_date': datetime.datetime(2021, 6, 27, 18, 29, 51)},
 'access': {'version': 'V1.1.3',
  'url': 'https://api.github.com/repos/pysal/access/tarball/V1.1.3',
  'release_date': datetime.datetime(2021, 1, 31, 21, 31, 11)},
 'esda': {'version': 'v2.4.1',
  'url': 'https://api.github.com/repos/pysal/esda/tarball/v2.4.1',
  'release_date': datetime.datetime(2021, 7, 27, 12, 54, 27)},
 'giddy': {'version': 'v2.3.3',
  'url': 'https://api.github.com/repos/pysal/giddy/tarball/v2.3.3',
  'release_date': datetime.datetime(2020, 6, 10, 4, 59, 45)},
 'inequality': {'version': 'v1.0.0',
  'url': 'https://api.github.com/repos/pysal/inequality/tarball/v1.0.0',
  'release_date': datetime.datetime(2018, 10, 31, 22, 28, 18)},
 'pointpats': {'version': 'v2.2.0',
  'url': 'https://api.github.com/repos/pysal/pointpats/tarball/v2.2.0',
  'release_date': datetime.datetime(2020, 7, 27, 

In [164]:
from datetime import datetime
datetime.fromtimestamp(0)
ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


final_pulls = {}
final_issues = {}
for package in packages:
    filtered_issues = []
    filtered_pulls = []
    released = github_releases[package]['release_date']
    package_pulls = pulls_closed[package]
    package_issues = issues_closed[package]
    for issue in package_issues:
        #print(issue['number'], issue['title'], issue['closed_at'])
        closed = datetime.strptime(issue['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_issues.append(issue)
    final_issues[package] = filtered_issues
    for pull in package_pulls:
        #print(pull['number'], pull['title'], pull['closed_at'])
        closed = datetime.strptime(pull['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_pulls.append(pull)
    final_pulls[package] = filtered_pulls
    print(package, released, len(package_issues), len(filtered_issues), len(package_pulls),
         len(filtered_pulls))

libpysal 2021-06-27 18:29:51 32 28 0 0
access 2021-01-31 21:31:11 0 0 0 0
esda 2021-07-27 12:54:27 20 20 0 0
giddy 2020-06-10 04:59:45 0 0 0 0
inequality 2018-10-31 22:28:18 0 0 0 0
pointpats 2020-07-27 22:17:33 6 0 1 0
segregation 2021-06-16 22:23:41 20 10 0 0
spaghetti 2021-06-28 23:54:51 60 55 0 0
mgwr 2020-09-08 21:20:34 11 0 1 0
spglm 2020-09-08 20:34:08 0 0 0 0
spint 2020-09-09 02:28:50 2 0 0 0
spreg 2021-06-29 19:21:48 12 10 5 3
spvcm 2020-02-02 19:42:39 0 0 0 0
tobler 2021-06-30 18:24:55 9 9 6 6
mapclassify 2021-07-27 03:06:32 6 6 2 2
splot 2021-07-27 15:04:13 10 9 4 3
spopt 2021-06-28 23:21:33 45 39 19 19
pysal 2021-07-31 12:00:00 4 4 0 0


In [165]:
issue_details = final_issues
pull_details = final_pulls

In [166]:
packages

{'libpysal': '4.5.1.post2',
 'access': '1.1.3',
 'esda': '2.4.1',
 'giddy': '2.3.3',
 'inequality': '1.0.0',
 'pointpats': '2.2.0',
 'segregation': '2.0.0',
 'spaghetti': '1.6.2',
 'mgwr': '2.1.2',
 'spglm': '1.0.8',
 'spint': '1.0.7',
 'spreg': '1.2.4',
 'spvcm': '0.3.0',
 'tobler': '0.8.2',
 'mapclassify': '2.4.3',
 'splot': '1.1.4',
 'spopt': '0.1.2',
 'pysal': '2.4.0'}

In [167]:
github_releases['pysal']['release_date'] = release_date

In [168]:
released

datetime.datetime(2021, 7, 31, 12, 0)

In [169]:
packages.keys()

dict_keys(['libpysal', 'access', 'esda', 'giddy', 'inequality', 'pointpats', 'segregation', 'spaghetti', 'mgwr', 'spglm', 'spint', 'spreg', 'spvcm', 'tobler', 'mapclassify', 'splot', 'spopt', 'pysal'])

In [170]:
spvcm = packages['spvcm']

In [171]:
## skip packages not released since last meta release

In [172]:
for package in github_releases:
    if github_releases[package]['release_date']>since:
        print("new: ",package)
    else:
        print('old:', package)

new:  libpysal
new:  access
new:  esda
old: giddy
old: inequality
old: pointpats
new:  segregation
new:  spaghetti
old: mgwr
old: spglm
old: spint
new:  spreg
old: spvcm
new:  tobler
new:  mapclassify
new:  splot
new:  spopt
new:  pysal


In [173]:
github_releases

{'libpysal': {'version': 'v4.5.1.post2',
  'url': 'https://api.github.com/repos/pysal/libpysal/tarball/v4.5.1.post2',
  'release_date': datetime.datetime(2021, 6, 27, 18, 29, 51)},
 'access': {'version': 'V1.1.3',
  'url': 'https://api.github.com/repos/pysal/access/tarball/V1.1.3',
  'release_date': datetime.datetime(2021, 1, 31, 21, 31, 11)},
 'esda': {'version': 'v2.4.1',
  'url': 'https://api.github.com/repos/pysal/esda/tarball/v2.4.1',
  'release_date': datetime.datetime(2021, 7, 27, 12, 54, 27)},
 'giddy': {'version': 'v2.3.3',
  'url': 'https://api.github.com/repos/pysal/giddy/tarball/v2.3.3',
  'release_date': datetime.datetime(2020, 6, 10, 4, 59, 45)},
 'inequality': {'version': 'v1.0.0',
  'url': 'https://api.github.com/repos/pysal/inequality/tarball/v1.0.0',
  'release_date': datetime.datetime(2018, 10, 31, 22, 28, 18)},
 'pointpats': {'version': 'v2.2.0',
  'url': 'https://api.github.com/repos/pysal/pointpats/tarball/v2.2.0',
  'release_date': datetime.datetime(2020, 7, 27, 

In [174]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
tag_dates = {}
for subpackage in packages:
    released = github_releases[subpackage]['release_date']
    tag_date = released.strftime("%Y-%m-%d")
    tag_dates[subpackage] = tag_date
    print(tag_date)
    #tag_date = tag_dates[subpackage]
    ncommits = 0
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        ncommits_total = len(check_output(cmd).splitlines())
    print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

2021-06-27
libpysal 57 48 2021-06-27
2021-01-31
access 6 0 2021-01-31
2021-07-27
esda 69 69 2021-07-27
2020-06-10
giddy 69 0 2020-06-10
2018-10-31
inequality 69 0 2018-10-31
2020-07-27
pointpats 69 0 2020-07-27
2021-06-16
segregation 57 57 2021-06-16
2021-06-28
spaghetti 171 149 2021-06-28
2020-09-08
mgwr 171 0 2020-09-08
2020-09-08
spglm 171 0 2020-09-08
2020-09-09
spint 171 0 2020-09-09
2021-06-29
spreg 18 15 2021-06-29
2020-02-02
spvcm 18 0 2020-02-02
2021-06-30
tobler 13 13 2021-06-30
2021-07-27
mapclassify 2 2 2021-07-27
2021-07-27
splot 18 18 2021-07-27
2021-06-28
spopt 130 103 2021-06-28
2021-07-31
pysal 2 2 2021-07-31


In [175]:
activity

{'libpysal': 48,
 'access': 0,
 'esda': 69,
 'giddy': 0,
 'inequality': 0,
 'pointpats': 0,
 'segregation': 57,
 'spaghetti': 149,
 'mgwr': 0,
 'spglm': 0,
 'spint': 0,
 'spreg': 15,
 'spvcm': 0,
 'tobler': 13,
 'mapclassify': 2,
 'splot': 18,
 'spopt': 103,
 'pysal': 2}

In [176]:
subpackage

'pysal'

In [177]:
CWD

'/Users/serge/Dropbox/p/pysal/src/pysal/tools'

In [178]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
for subpackage in packages:
    ncommits = 0
    tag_date = tag_dates[subpackage]
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        print(ncommits)
        ncommits_total = len(check_output(cmd).splitlines())
        print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

48
libpysal 57 48 2021-06-27
0
access 6 0 2021-01-31
69
esda 69 69 2021-07-27
57
segregation 57 57 2021-06-16
149
spaghetti 171 149 2021-06-28
15
spreg 18 15 2021-06-29
13
tobler 13 13 2021-06-30
2
mapclassify 2 2 2021-07-27
18
splot 18 18 2021-07-27
103
spopt 130 103 2021-06-28
2
pysal 2 2 2021-07-31


In [179]:
since_date

'--since="2021-01-31"'

In [180]:
cmd_until

['git', 'log', '--oneline', '--since="2021-01-31"', '--until="2021-07-31"']

In [181]:
identities = {'Levi John Wolf': ('ljwolf', 'Levi John Wolf'),
              'Serge Rey': ('Serge Rey', 'Sergio Rey', 'sjsrey', 'serge'),
              'Wei Kang': ('Wei Kang', 'weikang9009'),
              'Dani Arribas-Bel': ('Dani Arribas-Bel', 'darribas'),
              'Antti Härkönen': ( 'antth', 'Antti Härkönen', 'Antti Härkönen', 'Antth'  ),
              'Juan C Duque': ('Juan C Duque', "Juan Duque"),
              'Renan Xavier Cortes': ('Renan Xavier Cortes', 'renanxcortes', 'Renan Xavier Cortes'   ),
              'Taylor Oshan': ('Tayloroshan', 'Taylor Oshan', 'TaylorOshan'),
              'Tom Gertin': ('@Tomgertin', 'Tom Gertin', '@tomgertin')
}

def regularize_identity(string):
    string = string.decode()
    for name, aliases in identities.items():
        for alias in aliases:
            if alias in string:
                string = string.replace(alias, name)
    if len(string.split(' '))>1:
        string = string.title()
    return string.lstrip('* ')

In [182]:
author_cmd = ['git', 'log', '--format=* %aN', since_date]

In [183]:
author_cmd.append('blank')

In [184]:
author_cmd

['git', 'log', '--format=* %aN', '--since="2021-01-31"', 'blank']

In [185]:
from collections import Counter

In [186]:
tag_dates

{'libpysal': '2021-06-27',
 'access': '2021-01-31',
 'esda': '2021-07-27',
 'giddy': '2020-06-10',
 'inequality': '2018-10-31',
 'pointpats': '2020-07-27',
 'segregation': '2021-06-16',
 'spaghetti': '2021-06-28',
 'mgwr': '2020-09-08',
 'spglm': '2020-09-08',
 'spint': '2020-09-09',
 'spreg': '2021-06-29',
 'spvcm': '2020-02-02',
 'tobler': '2021-06-30',
 'mapclassify': '2021-07-27',
 'splot': '2021-07-27',
 'spopt': '2021-06-28',
 'pysal': '2021-07-31'}

In [187]:
authors_global = set()
authors = {}
global_counter = Counter()
counters = dict()
cmd = ['git', 'log', '--oneline', since_date]
total_commits = 0
activity = {}
for subpackage in packages:
    ncommits = 0
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        ncommits = len(check_output(cmd).splitlines())
        print(cmd)
        tag_date = tag_dates[subpackage]
        tag_date = (datetime.strptime(tag_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
        author_cmd[-1] = '--until="{tag_date}"'.format(tag_date=tag_date)
        #cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        print(subpackage, author_cmd)


        all_authors = check_output(author_cmd).splitlines()
        counter = Counter([regularize_identity(author) for author in all_authors])
        global_counter += counter
        counters.update({subpackage: counter})
        unique_authors = sorted(set(all_authors))
        authors[subpackage] =  unique_authors
        authors_global.update(unique_authors)
    total_commits += ncommits
    activity[subpackage] = ncommits

['git', 'log', '--oneline', '--since="2021-01-31"']
libpysal ['git', 'log', '--format=* %aN', '--since="2021-01-31"', '--until="2021-06-28"']
['git', 'log', '--oneline', '--since="2021-01-31"']
access ['git', 'log', '--format=* %aN', '--since="2021-01-31"', '--until="2021-02-01"']
['git', 'log', '--oneline', '--since="2021-01-31"']
esda ['git', 'log', '--format=* %aN', '--since="2021-01-31"', '--until="2021-07-28"']
['git', 'log', '--oneline', '--since="2021-01-31"']
segregation ['git', 'log', '--format=* %aN', '--since="2021-01-31"', '--until="2021-06-17"']
['git', 'log', '--oneline', '--since="2021-01-31"']
spaghetti ['git', 'log', '--format=* %aN', '--since="2021-01-31"', '--until="2021-06-29"']
['git', 'log', '--oneline', '--since="2021-01-31"']
spreg ['git', 'log', '--format=* %aN', '--since="2021-01-31"', '--until="2021-06-30"']
['git', 'log', '--oneline', '--since="2021-01-31"']
tobler ['git', 'log', '--format=* %aN', '--since="2021-01-31"', '--until="2021-07-01"']
['git', 'log'

In [188]:
author_cmd

['git',
 'log',
 '--format=* %aN',
 '--since="2021-01-31"',
 '--until="2021-08-01"']

In [189]:
subpackage

'pysal'

In [190]:
counter

Counter({'Serge Rey': 2})

In [191]:
authors_global

{b'* Arfon Smith',
 b'* Dani Arribas-Bel',
 b'* Elliott Sales de Andrade',
 b'* James Gaboardi',
 b'* Justin Pihony',
 b'* Leo Morales',
 b'* Levi John Wolf',
 b'* Martin Fleischmann',
 b'* MgeeeeK',
 b'* Pedro Amaral',
 b'* Pedro Camargo',
 b'* Serge Rey',
 b'* Sergio Rey',
 b'* Stefanie Lumnitz',
 b'* TLouf',
 b'* Xin (Selena) Feng',
 b'* Xin Feng',
 b'* dependabot[bot]',
 b'* eli knaap',
 b'* jeffcsauer',
 b'* jkoschinsky',
 b'* ljwolf'}

In [192]:
activity

{'libpysal': 57,
 'access': 6,
 'esda': 69,
 'giddy': 0,
 'inequality': 0,
 'pointpats': 0,
 'segregation': 57,
 'spaghetti': 171,
 'mgwr': 0,
 'spglm': 0,
 'spint': 0,
 'spreg': 18,
 'spvcm': 0,
 'tobler': 13,
 'mapclassify': 2,
 'splot': 18,
 'spopt': 130,
 'pysal': 2}

In [193]:
counters

{'libpysal': Counter({'James Gaboardi': 26,
          'Serge Rey': 22,
          'Elliott Sales De Andrade': 1,
          'Mgeeeek': 2,
          'Dani Arribas-Bel': 1}),
 'access': Counter({'Jkoschinsky': 1}),
 'esda': Counter({'James Gaboardi': 6,
          'Dependabot[Bot]': 1,
          'Serge Rey': 6,
          'Tlouf': 3,
          'Leo Morales': 1,
          'Levi John Wolf': 43,
          'Martin Fleischmann': 7,
          'Jeffcsauer': 1,
          'Dani Arribas-Bel': 1}),
 'segregation': Counter({'Eli Knaap': 49,
          'Serge Rey': 6,
          'James Gaboardi': 2}),
 'spaghetti': Counter({'James Gaboardi': 151,
          'Arfon Smith': 1,
          'Stefanie Lumnitz': 1}),
 'spreg': Counter({'James Gaboardi': 11, 'Serge Rey': 2, 'Pedro Amaral': 2}),
 'tobler': Counter({'Eli Knaap': 10, 'Serge Rey': 2, 'Martin Fleischmann': 1}),
 'mapclassify': Counter({'Serge Rey': 1, 'Justin Pihony': 1}),
 'splot': Counter({'Stefanie Lumnitz': 2,
          'James Gaboardi': 8,
         

In [194]:
counters

{'libpysal': Counter({'James Gaboardi': 26,
          'Serge Rey': 22,
          'Elliott Sales De Andrade': 1,
          'Mgeeeek': 2,
          'Dani Arribas-Bel': 1}),
 'access': Counter({'Jkoschinsky': 1}),
 'esda': Counter({'James Gaboardi': 6,
          'Dependabot[Bot]': 1,
          'Serge Rey': 6,
          'Tlouf': 3,
          'Leo Morales': 1,
          'Levi John Wolf': 43,
          'Martin Fleischmann': 7,
          'Jeffcsauer': 1,
          'Dani Arribas-Bel': 1}),
 'segregation': Counter({'Eli Knaap': 49,
          'Serge Rey': 6,
          'James Gaboardi': 2}),
 'spaghetti': Counter({'James Gaboardi': 151,
          'Arfon Smith': 1,
          'Stefanie Lumnitz': 1}),
 'spreg': Counter({'James Gaboardi': 11, 'Serge Rey': 2, 'Pedro Amaral': 2}),
 'tobler': Counter({'Eli Knaap': 10, 'Serge Rey': 2, 'Martin Fleischmann': 1}),
 'mapclassify': Counter({'Serge Rey': 1, 'Justin Pihony': 1}),
 'splot': Counter({'Stefanie Lumnitz': 2,
          'James Gaboardi': 8,
         

In [195]:
def get_tag(title, level="##", as_string=True):
    words = title.split()
    tag = "-".join([word.lower() for word in words])
    heading = level+" "+title
    line = "\n\n<a name=\"{}\"></a>".format(tag)
    lines = [line]
    lines.append(heading)
    if as_string:
        return "\n".join(lines)
    else:
        return lines

In [196]:
subs = issue_details.keys()
table = []
txt = []
lines = get_tag("Changes by Package", as_string=False)

for sub in subs:
    total= issue_details[sub]
    pr = pull_details[sub]
    
    row = [sub, activity[sub], len(total), len(pr)]
    table.append(row)
    #line = "\n<a name=\"{sub}\"></a>".format(sub=sub)
    #lines.append(line)
    #line = "### {sub}".format(sub=sub)
    #lines.append(line)
    lines.extend(get_tag(sub.lower(), "###", as_string=False))
    for issue in total:
        url = issue['html_url']
        title = issue['title']
        number = issue['number']
        line = "* [#{number}:]({url}) {title} ".format(title=title,
                                                     number=number,
                                                     url=url)
        lines.append(line)



In [197]:
sub

'pysal'

In [198]:
os.chdir(CWD)

import pandas

In [199]:
df = pandas.DataFrame(table, columns=['package', 'commits', 'total issues', 'pulls'])

In [200]:
df.head()

,package,commits,total issues,pulls
0,libpysal,57,28,0
1,access,6,0,0
2,esda,69,20,0
3,giddy,0,0,0
4,inequality,0,0,0


In [201]:
df.sort_values(['commits','pulls'], ascending=False)\
  .to_html('./commit_table.html', index=None)

In [202]:
df.sum()

package         libpysalaccessesdagiddyinequalitypointpatssegr...
commits                                                       543
total issues                                                  190
pulls                                                          33
dtype: object

In [203]:
contributor_table = pandas.DataFrame.from_dict(counters).fillna(0).astype(int).T

In [204]:
contributor_table.to_html('./contributor_table.html')

In [205]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits')

,commits
Arfon Smith,1
Dani Arribas-Bel,2
Dependabot[Bot],2
Eli Knaap,59
Elliott Sales De Andrade,1
James Gaboardi,283
Jeffcsauer,1
Jkoschinsky,1
Justin Pihony,1
Leo Morales,1


In [206]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits').to_html('./commits_by_person.html')

In [207]:
totals

James Gaboardi              283
Serge Rey                    51
Elliott Sales De Andrade      1
Mgeeeek                       2
Dani Arribas-Bel              2
Jkoschinsky                   1
Dependabot[Bot]               2
Tlouf                         3
Leo Morales                   1
Levi John Wolf               43
Martin Fleischmann           16
Jeffcsauer                    1
Eli Knaap                    59
Arfon Smith                   1
Stefanie Lumnitz              3
Pedro Amaral                  2
Justin Pihony                 1
Pedro Camargo                 1
Xin (Selena) Feng             8
Xin Feng                      5
dtype: int64

In [208]:
n_commits = df.commits.sum()
n_issues = df['total issues'].sum()
n_pulls = df.pulls.sum()

In [209]:
n_commits

543

In [210]:
#Overall, there were 719 commits that closed 240 issues, together with 105 pull requests across 12 packages since our last release on 2017-11-03.
#('{0} Here is a really long '
#           'sentence with {1}').format(3, 5))
line = ('Overall, there were {n_commits} commits that closed {n_issues} issues,'  
    ' together with {n_pulls} pull requests since our last release' 
        ' on {since_date}.\n'.format(n_commits=n_commits, n_issues=n_issues,
        n_pulls=n_pulls, since_date = start_date))

In [211]:
line

'Overall, there were 543 commits that closed 190 issues, together with 33 pull requests since our last release on 2021-01-31.\n'

## append html files to end of changes.md with tags for toc

In [227]:
with open('changes.md', 'w') as fout:
    fout.write(line)
    fout.write("\n".join(lines))
    fout.write(get_tag("Contributors"))
    fout.write("\n\nMany thanks to all of the following individuals who contributed to this release:\n\n")
    
    
    
    totals = contributor_table.sum(axis=0).T
    contributors = totals.index.values
    contributors.sort()
    contributors = contributors.tolist() 
    contributors = [ f'\n - {contributor}' for contributor in contributors]
    fout.write("".join(contributors))
    
